# Исследование способов определения контуров документов на фото
- Автор: Кирилл Киселев
- Дата начала: 13.04.2022
- Описание исследования: имеется 21 фото документов. Необходимо найти способы определения контуров документов на фото.

## 1. Имеющиеся изображения

## 2. Загрузка библиотек

In [1]:
# !pip install imutils==0.5.4

In [1]:
import cv2
import imutils
import numpy as np
import matplotlib as plt
import os
import math
from PIL import Image
from pyzbar import pyzbar

## 3. Глобальные переменные

In [2]:
!pwd

/home/kirillk/PycharmProjects/useful_notebooks/ocr


In [3]:
FILE_PATH = '/home/kirillk/datasets/OCR/ocr_test_cases_2/'

## 4. Необходимые функции

In [4]:
def detect_shape(contour):
    """Returns the shape (e.g. 'triangle', 'square') from the contour"""

    detected_shape = '-----'

    # Calculate perimeter of the contour:
    perimeter = cv2.arcLength(contour, True)

    # Get a contour approximation:
    contour_approx = cv2.approxPolyDP(contour, 0.03 * perimeter, True)

    # Check if the number of vertices is 3. In this case, the contour is a triangle
    if len(contour_approx) == 3:
        detected_shape = 'triangle'

    # Check if the number of vertices is 4. In this case, the contour is a square/rectangle
    elif len(contour_approx) == 4:

        # We calculate the aspect ration from the bounding rect:
        x, y, width, height = cv2.boundingRect(contour_approx)
        aspect_ratio = float(width) / height

        # A square has an aspect ratio close to 1 (comparison chaining is used):
        if 0.90 < aspect_ratio < 1.10:
            detected_shape = "square"
        else:
            detected_shape = "rectangle"

    # Check if the number of vertices is 5. In this case, the contour is a pentagon
    elif len(contour_approx) == 5:
        detected_shape = "pentagon"

    # Check if the number of vertices is 6. In this case, the contour is a hexagon
    elif len(contour_approx) == 6:
        detected_shape = "hexagon"

    # The shape as more than 6 vertices. In this example, we assume that is a circle
    else:
        detected_shape = "circle"

    # return the name of the shape and the found vertices
    return detected_shape, contour_approx

In [ ]:
def draw_contour_outline(img, cnts, color, thickness=1):
    """Draws contours outlines of each contour"""

    for cnt in cnts:
        cv2.drawContours(img, [cnt], 0, color, thickness)

In [ ]:
def find_contours(image):
    blurred = cv2.GaussianBlur(image, (3, 3), 0)
    T, thresh_img = cv2.threshold(blurred, 215, 255, 
                                  cv2.THRESH_BINARY)
    (_, cnts) = cv2.findContours(thresh_img, 
                                    cv2.RETR_EXTERNAL,
                                    cv2.CHAIN_APPROX_SIMPLE)
    return cnts

## 5. Данные

In [5]:
# список фото
photo_list = sorted([photo for photo in os.listdir(FILE_PATH) if photo.endswith('.jpg')])
photo_list

['001.jpg',
 '002.jpg',
 '003.jpg',
 '004.jpg',
 '005.jpg',
 '006.jpg',
 '007.jpg',
 '008.jpg',
 '009.jpg',
 '010.jpg',
 '011.jpg',
 '012.jpg',
 '013.jpg',
 '014.jpg',
 '015.jpg',
 '016.jpg',
 '017.jpg',
 '018.jpg',
 '019.jpg',
 '020.jpg',
 '021.jpg']

In [6]:
# посмотрим на фото

for photo in photo_list:
    image = cv2.imread(FILE_PATH + photo)
    cv2.imshow("Image", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

## 6. Определение контуров

In [22]:
im = cv2.imread(FILE_PATH + photo_list[0])
imgray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(imgray, 160, 255, 0)  # 160
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

max_contour = 0
num_of_contour = None

for num, i in enumerate(contours):
    if len(i) > max_contour:
        max_contour = len(i)
        num_of_contour = num

shape, vertices = detect_shape(contours[num_of_contour])
# print(vertices)
img = cv2.polylines(im, [vertices], True, (0,0,255), 5)
# img = cv2.resize(img, (0, 0), fx=0.5, fy=0.5)

cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [24]:
# посмотрим на все фото
for photo in photo_list:
    im = cv2.imread(FILE_PATH + photo)
    imgray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(imgray, 160, 255, 0)  # 160
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    max_contour = 0
    num_of_contour = None

    for num, i in enumerate(contours):
        if len(i) > max_contour:
            max_contour = len(i)
            num_of_contour = num

    shape, vertices = detect_shape(contours[num_of_contour])
    # print(vertices)
    img = cv2.polylines(im, [vertices], True, (0,0,255), 3)
    # img = cv2.resize(img, (0, 0), fx=0.5, fy=0.5)

    cv2.imshow("Image", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

### Вывод
Как видно изрезультатов работы скрипта - определить контуры документа на фото очень непросто.

На это влияет несколько факторов:
1. Фон, на котором сделано фото
2. Артефакты (руки)
3. Дополнительные объекты на фото. 

## 7. Определение контуров на качественных фото

Теперь поработаем с качественными фотографиями двух типов:

1. Качество съемки удовлетворительное
2. Качество съемки близко к идаельному в плане освещения и фона

In [7]:
FILE_PATH_2 = '/home/kirillk/datasets/OCR/ocr_test_cases_3/'

In [10]:
good_photo_list = sorted(os.listdir(FILE_PATH_2))
good_photo_list = [i for i in good_photo_list if i.endswith('.JPG')]
good_photo_list

['001.JPG',
 '002.JPG',
 '003.JPG',
 '004.JPG',
 '005.JPG',
 '006.JPG',
 '007.JPG',
 '008.JPG',
 '009.JPG',
 '010.JPG',
 '011.JPG',
 '012.JPG',
 '013.JPG']

In [11]:
# посмотрим на фото
for photo in good_photo_list:
    image = cv2.imread(FILE_PATH_2 + photo)
    image = cv2.resize(image, (0, 0), fx=0.2, fy=0.2)
    cv2.imshow("Image", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

error: OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [11]:
# посмотрим на фото с контурами
for photo in good_photo_list:
    im = cv2.imread(FILE_PATH_2 + photo)
    imgray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(imgray, 160, 255, 0)  # 160
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    max_contour = 0
    num_of_contour = None

    for num, i in enumerate(contours):
        if len(i) > max_contour:
            max_contour = len(i)
            num_of_contour = num

    shape, vertices = detect_shape(contours[num_of_contour])
    # print(vertices)
    img = cv2.polylines(im, [vertices], True, (0, 0, 255), 8)
    img = cv2.resize(img, (0, 0), fx=0.23, fy=0.23)

    cv2.imshow("Image", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

### Вывод
Определение контуров документа на качественном фото также зависит от фона и освещения.

## 8. Поворот фото

Для исследования возможностей поворота фотографий выделим из перечня фотографии со значтельным поворотом.

In [12]:
rotated_photo_list = ['009.JPG', '010.JPG', '011.JPG', '012.JPG']

In [13]:
# посмотрим на фото
for photo in rotated_photo_list:
    image = cv2.imread(FILE_PATH_2 + photo)
    image = cv2.resize(image, (0, 0), fx=0.2, fy=0.2)
    cv2.imshow("Image", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [14]:
# посмотрим на фото с контурами
for photo in rotated_photo_list:
    im = cv2.imread(FILE_PATH_2 + photo)
    imgray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(imgray, 160, 255, 0)  # 160
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    max_contour = 0
    num_of_contour = None

    for num, i in enumerate(contours):
        if len(i) > max_contour:
            max_contour = len(i)
            num_of_contour = num

    shape, vertices = detect_shape(contours[num_of_contour])
    # print(vertices)
    img = cv2.polylines(im, [vertices], True, (0, 0, 255), 8)
    img = cv2.resize(img, (0, 0), fx=0.23, fy=0.23)

    cv2.imshow("Image", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

#### Вывод
Контуры на всех фото выделяются качественно. 

### Для примера обработаем первое фото из списка.

In [15]:
im = cv2.imread(FILE_PATH_2 + rotated_photo_list[0])
imgray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(imgray, 160, 255, 0)  # 160
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

max_contour = 0
num_of_contour = None

for num, i in enumerate(contours):
    if len(i) > max_contour:
        max_contour = len(i)
        num_of_contour = num

shape, vertices = detect_shape(contours[num_of_contour])
print(shape, vertices)
img = cv2.polylines(im, [vertices], True, (0, 0, 255), 8)
img = cv2.resize(img, (0, 0), fx=0.23, fy=0.23)

cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

rectangle [[[ 843  297]]

 [[ 203 3192]]

 [[2312 3640]]

 [[2826  736]]]


In [16]:
x1 = (vertices[1][0][0])
y1 = (vertices[1][0][1])

x2 = (vertices[2][0][0])
y2 = (vertices[2][0][1])

xM = (vertices[1][0][0])
yM = (vertices[2][0][1])

print(x1, y2, x2, y2, xM, yM)

203 3640 2312 3640 203 3640


In [17]:
catet_1_M = abs(x2 - xM)
catet_M_2 = abs(yM - y1)

print(catet_1_M)
print(catet_M_2)

2109
448


In [18]:
tg_a = catet_M_2 / catet_1_M
angle = round(math.degrees(math.atan(tg_a)))
angle

12

In [19]:
test_img = cv2.imread(FILE_PATH_2 + rotated_photo_list[0])
test_img = cv2.resize(test_img, (0, 0), fx=0.23, fy=0.23)
rotated = imutils.rotate_bound(test_img, -angle)
cv2.imshow(f"Rotated by {angle} Degrees", rotated)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Тест автоматизации поворота и кропа первого фото из списка.

In [38]:
im = cv2.imread(FILE_PATH_2 + rotated_photo_list[3])
im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(im_gray, 160, 255, 0)  # 160
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

max_contour = 0
num_of_contour = None

for num, i in enumerate(contours):
    if len(i) > max_contour:
        max_contour = len(i)
        num_of_contour = num

shape, vertices = detect_shape(contours[num_of_contour])

x1 = (vertices[1][0][0])
y1 = (vertices[1][0][1])

x2 = (vertices[2][0][0])
y2 = (vertices[2][0][1])

xM = (vertices[1][0][0])
yM = (vertices[2][0][1])

catet_M_2 = abs(x2 - xM)
catet_1_M = abs(yM - y1)

tg_a = catet_1_M / catet_M_2
angle = round(math.degrees(math.atan(tg_a)))

# здесь осуществили поворот, rotate_bound поворачивает изображение по часовой стрелке
img = imutils.rotate_bound(im_gray, -angle)

# снова ищем контуры документа
_, thresh = cv2.threshold(img, 160, 255, 0)  
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

max_contour = 0
num_of_contour = None

for num, i in enumerate(contours):
    if len(i) > max_contour:
        max_contour = len(i)
        num_of_contour = num

_, vertices_2 = detect_shape(contours[num_of_contour])

x_1, x_2 = min([vertices_2[0][0][0], vertices_2[1][0][0]]), max([vertices_2[2][0][0], vertices_2[3][0][0]])
y_1, y_2 = min([vertices_2[0][0][1], vertices_2[3][0][1]]), max([vertices_2[2][0][1], vertices_2[2][0][1]])
print(x_1, x_2)
print(y_1, y_2)

# img = cv2.polylines(img, [vertices_2], True, (0, 0, 255), 8)
# print(vertices_2)

im = imutils.rotate_bound(im, -angle)

im = im[y_1:y_2, x_1:x_2]

im = cv2.resize(im, (0, 0), fx=0.3, fy=0.3)


# img = img[y_1:y_2, x_1:x_2]
# img = cv2.resize(img, (0, 0), fx=0.23, fy=0.23)

cv2.imshow("Image", im)
cv2.waitKey(0)
cv2.destroyAllWindows()

737 3675
833 2954


### Оптимизация скрипта поворота фото 

In [20]:
def find_contour(gray_image, threshold=160):
    _, thresh = cv2.threshold(gray_image, threshold, 255, 0)  # 160
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    max_contour = 0
    num_of_contour = None

    for num, i in enumerate(contours):
        if len(i) > max_contour:
            max_contour = len(i)
            num_of_contour = num
    return contours[num_of_contour]    

In [22]:
%%time
im = cv2.imread(FILE_PATH_2 + rotated_photo_list[0])
im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

doc_contour = find_contour(im_gray)
shape, vertices = detect_shape(doc_contour)

assert shape=='rectangle'

x1, y1 = vertices[1][0][0], vertices[1][0][1]
x2, y2 = vertices[2][0][0], vertices[2][0][1]
xM, yM = vertices[1][0][0], vertices[2][0][1]

# catet_M_2 = abs(x2 - xM)
# catet_1_M = abs(yM - y1)

catet_M_2 = (x2 - xM)
catet_1_M = (yM - y1)

tg_a = catet_1_M / catet_M_2
angle = round(math.degrees(math.atan(tg_a)))

# здесь осуществили поворот, rotate_bound поворачивает изображение по часовой стрелке
img = imutils.rotate_bound(im_gray, -angle)

# снова ищем контуры документа
contour_for_crop = find_contour(img)
_, vertices_2 = detect_shape(contour_for_crop)

x_1, x_2 = min([vertices_2[0][0][0], vertices_2[1][0][0]]), max([vertices_2[2][0][0], vertices_2[3][0][0]])
y_1, y_2 = min([vertices_2[0][0][1], vertices_2[3][0][1]]), max([vertices_2[2][0][1], vertices_2[2][0][1]])

im = imutils.rotate_bound(im, -angle)
im = im[y_1:y_2, x_1:x_2]
im = cv2.resize(im, (0, 0), fx=0.3, fy=0.3)

cv2.imshow("Image", im)
cv2.waitKey(0)
cv2.destroyAllWindows()

CPU times: user 1.16 s, sys: 617 ms, total: 1.78 s
Wall time: 42.5 s
